# Network diagrams for NL/GQL

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mat
import numpy as np
import networkx as nwkx
import collections

#### Define modes function to get all modes in an nx-ny grid

In [ ]:
def modes(nx,ny):
        
    i = 0
    x = []
    for p in np.arange(-nx,nx+1):
        for q in np.arange(-ny,ny+1):

            x.append((p,q))
            i += 1
    
    return x

In [ ]:
print(modes(1,1))

#### Define triads function to get all feasible triads in an nx-ny grid

In [ ]:
def triads(nx,ny):
        
    x = []
    for p in modes(nx,ny):
        for q in modes(nx,ny):
                        
            k = (p[0]+q[0],p[1]+q[1])
            
            if not(k[0]==0 and k[1]==0) and not(p[0]==0 and p[1]==0) and not(q[0]==0 and q[1]==0): # non-zeros
          
                if np.abs(k[0])<=nx and np.abs(k[1])<=ny: # remove out of bounds
                   
                    if p != q and q != k: # no self-interactions
                    
                        x.append((p,q,k)) # pxq -> k
    
    return x

In [ ]:
print(len(triads(1,1)))

In [ ]:
print(triads(1,1))

In [ ]:
((0,1),(0,-1),(0,0)) in triads(1,1)

In [ ]:
m = modes(1,1)[3]

print("Looking for mode: ", m)

for t in triads(1,1):
    if(m in t):
        print("Present in triad:", t)

#### Edges between modes and triads based on adjacency

In [ ]:
def edges(nx,ny,A):
        
    m = modes(nx,ny)
    
    e = []
    for t in triads(nx,ny):
        
        i = m.index(t[0])
        j = m.index(t[1])
        
        if(A[i,j] > 0.0):
            e.append((t[0],t))
            e.append((t[1],t))
            e.append((t[2],t))
    
    
    return e

In [ ]:
A = np.zeros((9,9))
A[2,3] = 1.0
e = edges(1,1,A)

In [ ]:
print(e)

#### Graph based on edge list between modes and triads

In [ ]:
def graph(nx,ny,A):

    m = modes(nx,ny)
    m.remove((0,0))

    t = triads(nx,ny)
    e = edges(nx,ny,A)

    bpg=nwkx.Graph()

    bpg.add_nodes_from(m,bipartite=0)
    bpg.add_nodes_from(t,bipartite=1)
    bpg.add_edges_from(e)

    dl = list(nwkx.connected_components(bpg))[1:]
    for d in dl:
        for n in d:
            bpg.remove_node(n)

    return bpg

#### Node positions in graph drawing

In [ ]:
def nodepos(G):
    
    l, r = nwkx.bipartite.sets(G)
    pos = {}

    pos.update((node, (0, index)) for index, node in enumerate(l))
    pos.update((node, (1, index)) for index, node in enumerate(r))

    return pos

#### Adjacency matrix labels

In [ ]:
def ax_adj(ax,nx,ny):
    
    l = []
    for m in modes(nx,ny):
        l.append(str(m))
    
    for a in ax:
        a.set_xticks(np.arange(0,(2*nx+1)*(2*ny+1),1))
        a.set_yticks(np.arange(0,(2*nx+1)*(2*ny+1),1))
        a.set_xticklabels(l,fontsize=14)
        a.set_yticklabels(l,fontsize=14)

### Draw fully connected bipartite network

In [ ]:
A = np.ones((9,9))
G = graph(1,1,A)
pos = nodepos(G)

fig,ax = plt.subplots(figsize=(7,7))

nwkx.draw(G,pos=pos,node_size=25,with_labels=True,width=0.75)

plt.show()

In [ ]:
f,g = nwkx.bipartite.sets(G)

fig,ax = plt.subplots(1,2,figsize=(12,5))

mdegs = G.degree(f) # dictionary node:degree 
mvals = sorted([d[1] for d in mdegs])
mhist = [mvals.count(x) for x in mvals]
ax[0].plot(mvals,mhist,'o-')
ax[0].set_xlabel('Degree')
ax[0].set_ylabel('Number of modes')


mdegs = G.degree(g) # dictionary node:degree 
mvals = sorted([d[1] for d in mdegs])
mhist = [mvals.count(x) for x in mvals]
ax[1].plot(mvals,mhist,'o-')
ax[1].set_xlabel('Degree')
ax[1].set_ylabel('Number of triads')


plt.show()

### Draw bipartite network based on  adjacency for 2x2 grid

In [ ]:
dn = "data/"

#### NL

In [ ]:
fn = dn+"adjacency_nl_2x2.npz"
nl = np.load(fn,allow_pickle=True) 

nx = 1
ny = 1

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

im = ax[0].imshow(np.abs(nl['A']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$|B_{mn}|$',fontsize=15)

im = ax[1].imshow(np.angle(nl['A']),cmap="RdBu_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'$\angle B_{mn}$',fontsize=15)

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(nl['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none",norm=mat.colors.LogNorm())

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(nl['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
A = nl['C']

G = graph(1,1,A)
pos = nodepos(G)

fig,ax = plt.subplots(figsize=(12,12))

nwkx.draw(G,pos=pos,node_size=25,with_labels=True,width=0.75)

plt.show()

#### QL

In [ ]:
fn = dn+"adjacency_ql_2x2.npz"
ql = np.load(fn,allow_pickle=True) 

nx = 1
ny = 1

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

im = ax[0].imshow(np.abs(ql['A']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$|B_{mn}|$',fontsize=15)

im = ax[1].imshow(np.angle(ql['A']),cmap="RdBu_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'$\angle B_{mn}$',fontsize=15)

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(ql['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(ql['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
A = ql['C']
G = graph(1,1,A)
pos = nodepos(G)

fig,ax = plt.subplots(figsize=(7,7))

nwkx.draw(G,pos=pos,node_size=25,with_labels=True,width=0.75)

plt.show()

### Draw bipartite network based on  adjacency for 3x3 grid

In [ ]:
fn = dn+"adjacency_nl_3x3.npz"
nl = np.load(fn,allow_pickle=True) 

nx = 2
ny = 2

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(nl['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(nl['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
A = nl['C'] # test a fully connected graph

G = graph(2,2,A)
pos = nodepos(G)

fig,ax = plt.subplots(figsize=(7,7))

nwkx.draw(G,pos=pos,node_size=25,with_labels=False,width=0.75)

plt.show()

In [ ]:
f,g = nwkx.bipartite.sets(G)

fig,ax = plt.subplots(1,2,figsize=(12,5))

mdegs = G.degree(f) # dictionary node:degree 
mvals = sorted([d[1] for d in mdegs])
mhist = [mvals.count(x) for x in mvals]
ax[0].plot(mvals,mhist,'o-')
ax[0].set_xlabel('Degree')
ax[0].set_ylabel('Number of modes')


mdegs = G.degree(g) # dictionary node:degree 
mvals = sorted([d[1] for d in mdegs])
mhist = [mvals.count(x) for x in mvals]
ax[1].plot(mvals,mhist,'o-')
ax[1].set_xlabel('Degree')
ax[1].set_ylabel('Number of triads')


plt.show()

In [ ]:
fn = dn+"adjacency_ql_3x3.npz"
ql = np.load(fn,allow_pickle=True) 

nx = 2
ny = 2

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(ql['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(ql['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
A = ql['C'] # test a fully connected graph

G = graph(2,2,A)
pos = nodepos(G)

fig,ax = plt.subplots(figsize=(8,8))

nwkx.draw(G,pos=pos,node_size=25,with_labels=False,width=0.75)

plt.show()

In [ ]:
f,g = nwkx.bipartite.sets(G)

fig,ax = plt.subplots(1,2,figsize=(12,5))

mdegs = G.degree(f) # dictionary node:degree 
mvals = sorted([d[1] for d in mdegs])
mhist = [mvals.count(x) for x in mvals]
ax[0].plot(mvals,mhist,'o-')
ax[0].set_xlabel('Degree')
ax[0].set_ylabel('Number of modes')


mdegs = G.degree(g) # dictionary node:degree 
mvals = sorted([d[1] for d in mdegs])
mhist = [mvals.count(x) for x in mvals]
ax[1].plot(mvals,mhist,'o-')
ax[1].set_xlabel('Degree')
ax[1].set_ylabel('Number of triads')


plt.show()

In [ ]:
fn = dn+"adjacency_gql1_3x3.npz"
gql = np.load(fn,allow_pickle=True) 

nx = 2
ny = 2

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(gql['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(gql['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
A = gql['C'] # test a fully connected graph

G = graph(2,2,A)
pos = nodepos(G)

fig,ax = plt.subplots(figsize=(8,8))

nwkx.draw(G,pos=pos,node_size=25,with_labels=False,width=0.75)

plt.show()

In [ ]:
f,g = nwkx.bipartite.sets(G)

fig,ax = plt.subplots(1,2,figsize=(12,5))

mdegs = G.degree(f) # dictionary node:degree 
mvals = sorted([d[1] for d in mdegs])
mhist = [mvals.count(x) for x in mvals]
ax[0].plot(mvals,mhist,'o-')
ax[0].set_xlabel('Degree')
ax[0].set_ylabel('Number of modes')


mdegs = G.degree(g) # dictionary node:degree 
mvals = sorted([d[1] for d in mdegs])
mhist = [mvals.count(x) for x in mvals]
ax[1].plot(mvals,mhist,'o-')
ax[1].set_xlabel('Degree')
ax[1].set_ylabel('Number of triads')


plt.show()

In [ ]:
fn = dn+"adjacency_nl_2x2.npz"
nl = np.load(fn,allow_pickle=True) 

nx = 1
ny = 1

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

im = ax[0].imshow(np.abs(nl['A']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$|B_{mn}|$',fontsize=15)

im = ax[1].imshow(np.angle(nl['A']),cmap="RdBu_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'$\angle B_{mn}$',fontsize=15)

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(nl['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(nl['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

# ax_adj(ax,nx,ny)

plt.show()

In [ ]:
nx = 5
ny = 5

fn = dn+"adjacency_nl_6x6.npz"
nl = np.load(fn,allow_pickle=True) 

fn = dn+"adjacency_ql_6x6.npz"
ql = np.load(fn,allow_pickle=True) 

fn = dn+"adjacency_gql1_6x6.npz"
gql = np.load(fn,allow_pickle=True) 


G_nl = graph(nx,ny,nl['C'])
G_ql = graph(nx,ny,ql['C'])
G_gql = graph(nx,ny,gql['C'])

fig,ax = plt.subplots(1,3,figsize=(15,5))

f_nl,g_nl = nwkx.bipartite.sets(G_nl)
mdegs = G_nl.degree(f_nl) # dictionary node:degree 
mvals = sorted([d[1] for d in mdegs])
mhist = [mvals.count(x) for x in mvals]
ax[0].plot(mvals,mhist,'o-')
ax[0].set_title('NL')

f_ql,g_ql = nwkx.bipartite.sets(G_ql)
mdegs = G_ql.degree(f_ql) # dictionary node:degree 
mvals = sorted([d[1] for d in mdegs])
mhist = [mvals.count(x) for x in mvals]
ax[1].plot(mvals,mhist,'o-')
ax[1].set_title('QL')


f_gql,g_gql = nwkx.bipartite.sets(G_gql)
mdegs = G_gql.degree(f_gql) # dictionary node:degree 
mvals = sorted([d[1] for d in mdegs])
mhist = [mvals.count(x) for x in mvals]
ax[2].plot(mvals,mhist,'o-')
ax[2].set_title('GQL(1)')

for a in ax:
    a.set_xlabel('Degree')
    a.set_ylabel('Number of modes')


plt.show()

In [ ]:
fn = dn+"adjacency_gql1_6x6.npz"
gql = np.load(fn,allow_pickle=True) 

nx = 5
ny = 5

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(gql['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(gql['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

# ax_adj(ax,nx,ny)

plt.show()

In [ ]:
A = gql['C'] # test a fully connected graph
G = graph(5,5,A)

In [ ]:
f,g = nwkx.bipartite.sets(G)

fig,ax = plt.subplots(1,2,figsize=(12,5))

mdegs = G.degree(f) # dictionary node:degree 
mvals = sorted([d[1] for d in mdegs])
mhist = [mvals.count(x) for x in mvals]
ax[0].plot(mvals,mhist,'o-')
ax[0].set_xlabel('Degree')
ax[0].set_ylabel('Number of modes')


mdegs = G.degree(g) # dictionary node:degree 
mvals = sorted([d[1] for d in mdegs])
mhist = [mvals.count(x) for x in mvals]
ax[1].plot(mvals,mhist,'o-')
ax[1].set_xlabel('Degree')
ax[1].set_ylabel('Number of triads')


plt.show()

### Time dependent adjacency matrices

In [ ]:
nx,ny = 7,7
nl_sol = np.load(dn+"nl_sol_m53_e001_8x8.npz",allow_pickle=True) 
nl_adj = np.load(dn+"nl_adj_m53_e001_8x8.npz",allow_pickle=True) 

In [ ]:
fig,ax = plt.subplots()

ax.imshow(nl_sol['F'])

plt.show()

> this shows the mode and its conjugate being forced

In [ ]:
fig,ax = plt.subplots()

ax.set_title(f'NL',fontsize=10)
for i,x in enumerate(nl_sol['Emt']):
    ax.plot(nl_sol['t'],x,label=i)

plt.legend()
plt.yscale('log')
plt.show()

> this shows the evolution of energy in zonal modes

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(nl_adj['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(nl_adj['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

# ax_adj(ax,nx,ny)

plt.show()

> that's the NL solution adjacencies at t = 2000

In [ ]:
nx,ny = 7,7
ql_sol = np.load(dn+"ql_sol_m53_e001_8x8.npz",allow_pickle=True) 
ql_adj = np.load(dn+"ql_adj_m53_e001_8x8.npz",allow_pickle=True) 

In [ ]:
fig,ax = plt.subplots()

ax.set_title(f'$E_m$ in QL',fontsize=10)
for i,x in enumerate(ql_sol['Emt']):
    ax.plot(ql_sol['t'],x,label=i)

plt.legend()
plt.yscale('log')
plt.ylim(1e-12,1e2)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(ql_adj['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(ql_adj['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
nx,ny = 7,7
gql_sol = np.load(dn+"gql1_sol_m53_e001_8x8.npz",allow_pickle=True) 
gql_adj = np.load(dn+"gql1_adj_m53_e001_8x8.npz",allow_pickle=True) 

In [ ]:
fig,ax = plt.subplots()

ax.set_title(f'$E_m$ in GQL(1)',fontsize=10)
for i,x in enumerate(gql_sol['Emt']):
    ax.plot(gql_sol['t'],x,label=i)

plt.legend()
plt.yscale('log')
plt.ylim(1e-12,1e2)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(gql_adj['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(gql_adj['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

# ax_adj(ax,nx,ny)

plt.show()

In [ ]:
nx,ny = 7,7
nl_sol = np.load(dn+"nl_sol_e001_8x8.npz",allow_pickle=True) 
nl_adj = np.load(dn+"nl_adj_e001_8x8.npz",allow_pickle=True) 

In [ ]:
fig,ax = plt.subplots()

ax.imshow(nl_sol['F'])

plt.show()

> this is a band type forcing - forcing all modes wi"thin a zonal wavenumber

In [ ]:
fig,ax = plt.subplots()

ax.set_title(f'NL/GQL(M)',fontsize=10)
for i,x in enumerate(nl_sol['Emt']):
    ax.plot(nl_sol['t'],x,label=i)

plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

im = ax[0].imshow(np.abs(nl['A']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$|B_{mn}|$',fontsize=15)

im = ax[1].imshow(np.angle(nl['A']),cmap="RdBu_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'$\angle B_{mn}$',fontsize=15)

# ax_adj(ax,nx,ny)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(nl_adj['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(nl_adj['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

# ax_adj(ax,nx,ny)

plt.show()

In [ ]:
nx,ny = 7,7
ql_sol = np.load(dn+"ql_sol_e001_8x8.npz",allow_pickle=True) 
ql_adj = np.load(dn+"ql_adj_e001_8x8.npz",allow_pickle=True) 

In [ ]:
fig,ax = plt.subplots()

ax.set_title(f'$E_m$ in QL',fontsize=10)
for i,x in enumerate(ql_sol['Emt']):
    ax.plot(ql_sol['t'],x,label=i)

plt.legend()
plt.yscale('log')
plt.ylim(1e-12,1e2)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(ql_adj['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(ql_adj['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

# ax_adj(ax,nx,ny)

plt.show()

In [ ]:
nx,ny = 7,7
gql_sol = np.load(dn+"gql1_sol_e001_8x8.npz",allow_pickle=True) 
gql_adj = np.load(dn+"gql1_adj_e001_8x8.npz",allow_pickle=True) 

In [ ]:
fig,ax = plt.subplots()

ax.set_title(f'E_m in GQL',fontsize=10)
for i,x in enumerate(gql_sol['Emt']):
    ax.plot(gql_sol['t'],x,label=i)

plt.legend()
plt.yscale('log')
plt.ylim(1e-12,1e2)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(gql_adj['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(gql_adj['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

# ax_adj(ax,nx,ny)

plt.show()